In [5]:
# Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from scipy.interpolate import interp1d
import seaborn.objects as so
from scipy.stats import chi2_contingency

In [6]:
cancellations_hb_fp = os.path.join(os.getcwd(), "datasets", "cancellations_by_hb.csv")
cancellations_hb = pd.read_csv(cancellations_hb_fp)

population_fp = os.path.join(os.getcwd(), "datasets", "mid-year-pop-est-21-figures.xlsx")
population = pd.read_excel(population_fp)


In [7]:
population

,"Figures for Mid-Year Population Estimates for Scotland, mid-2021"
0,This spreadsheet contains the charts and their...
1,Source: National Records of Scotland (NRS)
2,Publication date
3,The data was published at 9:30am on 13 July 2022.
4,Geographic coverage
5,Scotland and its 32 council areas and 14 NHS b...
6,Time period
7,30 June 2011 to 30 June 2021
8,Supplier
9,National Records of Scotland (NRS)
